In [9]:
from ollama import chat
from pydantic import BaseModel
import pandas as pd
from gnews import GNews
import time
from typing import List, Dict, Any
from tenacity import retry, stop_after_attempt, wait_exponential

class NewsAnalysis(BaseModel):
    sentiment: str
    future_looking: bool

class NewsArticle(BaseModel):
    title: str
    sentiment: str
    future_looking: bool

def fetch_news(keyword: str, max_results: int = 6) -> List[Dict[str, Any]]:
    """Fetch news articles with error handling."""
    try:
        google_news = GNews(language='en', country='US', period='7d')
        news = google_news.get_news(keyword)
        return news[:max_results]
    except Exception as e:
        print(f"Error fetching news: {str(e)}")
        return []

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def analyze_sentiment(title: str) -> NewsAnalysis:
    """Analyze sentiment with retry logic for failed connections."""
    try:
        response = chat(
            messages=[{
                'role': 'user',
                'content': f"""Analyze the following title for sentiment (positive, negative, or neutral) 
                           and whether it provides future-looking financial insight, predictions, or 
                           guidance on whether to buy/hold/sell the stock (True or False): {title}"""
            }],
            model='llama2',  # Changed to llama2 as it's more commonly available
            format=NewsAnalysis.model_json_schema(),
        )
        
        return NewsAnalysis.model_validate_json(response['message']['content'])
    except Exception as e:
        print(f"Error analyzing sentiment for title: {title}")
        print(f"Error details: {str(e)}")
        raise  # Re-raise for retry mechanism

def main():
    # Fetch news
    news_articles = fetch_news("NVDA")
    if not news_articles:
        print("No news articles found.")
        return
    
    results = []
    
    # Process each article
    for article in news_articles:
        title = article['title']
        try:
            # Add delay to prevent rate limiting
            time.sleep(1)
            
            sentiment_analysis = analyze_sentiment(title)
            
            results.append(NewsArticle(
                title=title,
                sentiment=sentiment_analysis.sentiment,
                future_looking=sentiment_analysis.future_looking
            ))
            
        except Exception as e:
            print(f"Failed to analyze article: {title}")
            print(f"Error: {str(e)}")
            continue
    
    # Create DataFrame
    if results:
        df = pd.DataFrame([article.model_dump() for article in results])
        print("\nSentiment Analysis Results:")
        print(df)
        return df
    else:
        print("No results to display.")
        return pd.DataFrame()

if __name__ == "__main__":
    main()

Error analyzing sentiment for title: NVDA Stock Quote Price and Forecast - CNN
Error details: [WinError 10061] No connection could be made because the target machine actively refused it
Error analyzing sentiment for title: NVDA Stock Quote Price and Forecast - CNN
Error details: [WinError 10061] No connection could be made because the target machine actively refused it
Error analyzing sentiment for title: NVDA Stock Quote Price and Forecast - CNN
Error details: [WinError 10061] No connection could be made because the target machine actively refused it
Failed to analyze article: NVDA Stock Quote Price and Forecast - CNN
Error: RetryError[<Future at 0x2580861e610 state=finished raised ConnectError>]
Error analyzing sentiment for title: Jim Cramer Reveals the Next Best Time to Buy NVIDIA (NVDA) Stock - Yahoo Finance
Error details: [WinError 10061] No connection could be made because the target machine actively refused it
Error analyzing sentiment for title: Jim Cramer Reveals the Next Bes

KeyboardInterrupt: 